# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 43 new papers today
          17 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/16 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.11313


extracting tarball to tmp_2212.11313...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.11319


extracting tarball to tmp_2212.11319...

...

extracting tarball to tmp_2212.11940...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.11941


extracting tarball to tmp_2212.11941...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.11960


/tmp/ipykernel_2071/4030337529.py:34: LatexWarning: 2212.11960 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11425-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11425) | **Multi-Scale Stellar Associations across the Star Formation Hierarchy in  PHANGS-HST Nearby Galaxies: Methodology and Properties**  |
|| Kirsten L. Larson, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>James Lilly</mark>, <mark>Frank Bigiel</mark>, <mark>Kathryn Kreckel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-23*|
|*Comments*| *Submitted to MNRAS. Referee report received with minor comments, and "request to clarify if the smaller associations are always included in the larger ones and how this may affect the photometric fitting of the larger association if the groups have different ages." Revision in progress*|
|**Abstract**| We develop a method to identify and determine the physical properties of stellar associations using Hubble Space Telescope (HST) NUV-U-B-V-I imaging of nearby galaxies from the PHANGS-HST survey. We apply a watershed algorithm to density maps constructed from point source catalogues Gaussian smoothed to multiple physical scales from 8 to 64 pc. We develop our method on two galaxies that span the distance range in the PHANGS-HST sample: NGC 3351 (10 Mpc), NGC 1566 (18 Mpc). We test our algorithm with different parameters such as the choice of detection band for the point source catalogue (NUV or V), source density image filtering methods, and absolute magnitude limits. We characterise the properties of the resulting multi-scale associations, including sizes, number of tracer stars, number of associations, photometry, as well as ages, masses, and reddening from Spectral Energy Distribution fitting. Our method successfully identifies structures that occupy loci in the UBVI colour-colour diagram consistent with previously published catalogues of clusters and associations. The median ages of the associations increases from log(age/yr) = 6.6 to log(age/yr) = 6.9 as the spatial scale increases from 8 pc to 64 pc for both galaxies. We find that the youngest stellar associations, with ages < 3 Myr, indeed closely trace H ii regions in H$\alpha$ imaging, and that older associations are increasingly anti-correlated with the H$\alpha$ emission. Owing to our new method, the PHANGS-HST multi-scale associations provide a far more complete census of recent star formation activity than found with previous cluster and compact association catalogues. The method presented here will be applied to the full sample of 38 PHANGS-HST galaxies. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11319-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11319) | **The Dark Energy Survey Year 3 and eBOSS: constraining galaxy intrinsic  alignments across luminosity and colour space**  |
|| S. Samuroff, et al. -- incl., <mark>I. Harrison</mark>, <mark>K. Herner</mark> |
|*Appeared on*| *2022-12-23*|
|*Comments*| *Submitted to MNRAS; 22 pages, 11 figures*|
|**Abstract**| We present direct constraints on galaxy intrinsic alignments using the Dark Energy Survey Year 3 (DES Y3), the Extended Baryon Oscillation Spectroscopic Survey (eBOSS) and its precursor, the Baryon Oscillation Spectroscopic Survey (BOSS). Our measurements incorporate photometric red sequence (redMaGiC) galaxies from DES with median redshift $z\sim0.2-1.0$, luminous red galaxies (LRGs) from eBOSS at $z\sim0.8$, and also a SDSS-III BOSS CMASS sample at $z\sim0.5$. We measure two point intrinsic alignment correlations, which we fit using a model that includes lensing, magnification and photometric redshift error. Fitting on scales $6<r_{\rm p} < 70$ Mpc$/h$, we make a detection of intrinsic alignments in each sample, at $5\sigma-22\sigma$ (assuming a simple one parameter model for IAs). Using these red samples, we measure the IA-luminosity relation. Our results are statistically consistent with previous results, but offer a significant improvement in constraining power, particularly at low luminosity. With this improved precision, we see detectable dependence on colour between broadly defined red samples. It is likely that a more sophisticated approach than a binary red/blue split, which jointly considers colour and luminosity dependence in the IA signal, will be needed in future. We also compare the various signal components at the best fitting point in parameter space for each sample, and find that magnification and lensing contribute $\sim2-18\%$ of the total signal. As precision continues to improve, it will certainly be necessary to account for these effects in future direct IA measurements. Finally, we make equivalent measurements on a sample of Emission Line Galaxies (ELGs) from eBOSS at $z\sim 0.8$. We report a null detection, constraining the IA amplitude (assuming the nonlinear alignment model) to be $A_1=0.07^{+0.32}_{-0.42}$ ($|A_1|<0.78$ at $95\%$ CL). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11313-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11313) | **Radio jet-ISM interaction and positive radio-mechanical feedback in  Abell 1795**  |
|| Prathamesh D. Tamhane, et al. -- incl., <mark>Helen R. Russell</mark> |
|*Appeared on*| *2022-12-23*|
|*Comments*| *Accepted for publication in MNRAS. 22 pages, 13 figures 1 table*|
|**Abstract**| We present XSHOOTER observations with previous ALMA, MUSE and $HST$ observations to study the nature of radio-jet triggered star formation and the interaction of radio jets with the interstellar medium in the brightest cluster galaxy (BCG) in the Abell 1795 cluster. Using $HST$ UV data we determined an ongoing star formation rate of 9.3 M$_\odot$ yr$^{-1}$. The star formation follows the global Kennicutt-Schmidt law, however, it has a low efficiency compared to circumnuclear starbursts in nearby galaxies with an average depletion time of $\sim$1 Gyr. The star formation and molecular gas are offset by $\sim1$ kpc indicating that stars have decoupled from the gas. We detected an arc of high linewidth in ionized gas where electron densities are elevated by a factor of $\sim$4 suggesting a shock front driven by radio jets or peculiar motion of the BCG. An analysis of nebular emission line flux ratios suggests that the gas is predominantly ionized by star formation with a small contribution from shocks. We also calculated the velocity structure function (VSF) of the ionized and molecular gases using velocity maps to characterize turbulent motion in the gas. The ionized gas VSF suggests that the radio jets are driving supersonic turbulence in the gas. Thus radio jets can not only heat the atmosphere on large scales and may quench star formation on longer timescales while triggering star formation in positive feedback on short timescales of a few million years. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11639-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11639) | **Recovering the very metal-poor stars in the Gaia DR3 GSP-Spec catalog**  |
|| Tadafumi Matsuno, Else Starkenburg, Eduardo Balbinot, <mark>Amina Helmi</mark> |
|*Appeared on*| *2022-12-23*|
|*Comments*| *The catalog will be publicly made available when accepted, or before upon reasonable request*|
|**Abstract**| Context: In the latest Gaia data release (DR3), the GSP-Spec module has provided stellar parameters and chemical abundances measured from the RVS spectra alone. However, very metal-poor stars (VMP stars; $[\mathrm{Fe/H}]<-2$) suffer from parameter degeneracy due to a lack of information in their spectra, making it difficult to obtain reliable stellar parameters and metallicities for many of them. Aim: We aim to improve metallicity estimates for VMP stars analysed by the GSP-Spec module. Methods: We compute the Ca triplet equivalent widths from the published set of GSP-Spec stellar parameters. We then convert these obtained equivalent widths to metallicities adopting photometric temperatures and surface gravities that we derive based on Gaia and 2MASS catalogs. Results: Comparison to high-resolution studies shows that our approach drastically reduces the cases where the estimated metallicities are far off for VMP stars. Now only $23\%$ of VMP stars have a metallicity different by more than $0.5\,\mathrm{dex}$ from a high-resolution value, while this fraction is $76\%$ with the original metallicity estimates by the GSP-Spec module. We explore possible ways to remove stars with poor metallicity estimates while keeping as many stars with reliable metallicity as possible. Our improved metallicity estimates and new quality cut result in producing a catalog of bright ($G\lesssim 13$) metal-poor stars, containing 57 stars at $[\mathrm{Fe/H}]<-3$ and 2202 VMP stars. These numbers increase to 174 and 2794 if one allows a low level of metal-rich contaminants. Conclusion: The inclusion of photometric information greatly contributes to breaking parameter degeneracy, enabling precise metallicity estimates for VMP stars from Gaia RVS spectra. We produce a publicly-available catalog of bright metal-poor stars suitable for high-resolution follow-up. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11940-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11940) | **A Model for the Squeezed Bispectrum in the Non-Linear Regime**  |
|| Matteo Biagetti, et al. -- incl., <mark>Lina Castiblanco</mark> |
|*Appeared on*| *2022-12-23*|
|*Comments*| *19 pages, 5 figures*|
|**Abstract**| We present a model for the squeezed dark matter bispectrum, where the short modes are deep in the non-linear regime. We exploit the consistency relations for large-scale structures combined with a response function approach to write the squeezed bispectrum in terms of a few unknown functions of the short modes. We provide an ansatz for a fitting function for these response functions, checking that the resulting model is reliable when compared to the one-loop squeezed bispectrum. We then test the model against measured bispectra from numerical simulations for short modes ranging between $k \sim 0.1 \, h/$Mpc, and $k \sim 0.7 \, h/$Mpc at redshift $z=0$. To evaluate the goodness of the fit of our model we implement a non-Gaussian covariance and find agreement within $1$-$\sigma$ standard deviation of the simulated data. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11941-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11941) | **Sub-arcminute localization of 13 repeating fast radio bursts detected by  CHIME/FRB**  |
|| Daniele Michilli, et al. -- incl., <mark>Jane Kaczmarek</mark> |
|*Appeared on*| *2022-12-23*|
|*Comments*| **|
|**Abstract**| We report on improved sky localizations of thirteen repeating fast radio bursts (FRBs) discovered by CHIME/FRB via the use of interferometric techniques on channelized voltages from the telescope. These so-called 'baseband localizations' improve the localization uncertainty area presented in past studies by more than three orders of magnitude. The improved localization regions are provided for the full sample of FRBs to enable follow-up studies. The localization uncertainties, together with limits on the source distances from their dispersion measures (DMs), allow us to identify likely host galaxies for two of the FRB sources. FRB 20180814A lives in a massive passive red spiral at z~0.068 with very little indication of star formation, while FRB 20190303A resides in a merging pair of spiral galaxies at z~0.064 undergoing significant star formation. These galaxies show very different characteristics, further confirming the presence of FRB progenitors in a variety of environments even among the repeating sub-class. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11421-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11421) | **PACMAN: A pipeline to reduce and analyze Hubble Wide Field Camera 3 IR  Grism data**  |
|| <mark>Sebastian Zieba</mark>, <mark>Laura Kreidberg</mark> |
|*Appeared on*| *2022-12-23*|
|*Comments*| *8 pages, 2 figures, Published in JOSS, GitHub: this https URL*|
|**Abstract**| Here we present PACMAN, an end-to-end pipeline developed to reduce and analyze HST/WFC3 data. The pipeline includes both spectral extraction and light curve fitting. The foundation of PACMAN has been already used in numerous publications (e.g., Kreidberg et al., 2014; Kreidberg et al., 2018) and these papers have already accumulated hundreds of citations. The Hubble Space Telescope (HST) has become the preeminent workhorse facility for the characterization of extrasolar planets. HST currently has two of the most powerful space-based tools for characterizing exoplanets over a broad spectral range: The Space Telescope Imaging Spectrograph (STIS) in the UV and the Wide Field Camera 3 (WFC3) in the Near Infrared. With the introduction of a spatial scan mode on WFC3 where the star moves perpendicular to the dispersion direction during an exposure, WFC3 observations have become very efficient due to the reduction of overhead time and the possibility of longer exposures without saturation. For exoplanet characterization, WFC3 is used for transit and secondary eclipse spectroscopy, and phase curve observations. The instrument has two different grisms: G102 with a spectral range from 800 nm to up to 1150 nm and G141 encompassing 1075 nm to about 1700 nm. The spectral range of WFC3/G141 is primarily sensitive to molecular absorption from water at approximately 1.4 microns. This led to the successful detection of water in the atmosphere of over a dozen of exoplanets. The bluer part of WFC3, the G102 grism, is also sensitive to water and most notably led to the first detection of a helium exosphere. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.11960-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.11960) | **Gravitational wave lensing as a probe of halo properties and dark matter**  |
|| Giovanni Tambalo, Miguel Zumalacárregui, <mark>Liang Dai</mark>, Mark Ho-Yeuk Cheung |
|*Appeared on*| *2022-12-23*|
|*Comments*| *41 pages, 27 figures*|
|**Abstract**| Just like light, gravitational waves (GWs) are deflected and magnified by gravitational fields as they propagate through the Universe. However, their low frequency, phase coherence and feeble coupling to matter allow for distinct lensing phenomena, such as diffraction and central images, that are challenging to observe through electromagnetic sources. Here we explore how these phenomena can be used to probe features of gravitational lenses. We focus on two variants of the singular isothermal sphere, with 1) a variable slope of the matter density and 2) a central core. We describe the imprints of these features in the wave- and geometric-optics regimes, including the prospect of detecting central images. We forecast the capacity of LISA and advanced LIGO to study strongly lensed signals and measure the projected lens mass, impact parameter and slope or core size. A broad range of lens masses allows all parameters to be measured with precision up to $\sim 1/{\rm SNR}$, despite large degeneracies. Thanks to wave-optics corrections, all parameters can be measured, even when no central image forms. Although GWs are sensitive to projected quantities, we compute the probability distribution of lens redshift, virial mass and projection scale given a cosmology. As an application, we consider the prospect of constraining self-interacting and ultra-light dark matter, showing the regions of parameter space accessible to strongly-lensed GWs. The distinct GW signatures will enable novel probes of fundamental physics and astrophysics, including the properties of dark matter and the central regions of galactic halos. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.11425.md
    + _build/html/tmp_2212.11425/./lowRes_figs/NGC3351_reg_img.jpg
    + _build/html/tmp_2212.11425/./ngc3351_dolphot_color_vi_ub_ws_Mv_ALL.png
    + _build/html/tmp_2212.11425/./lowRes_figs/NGC1566_reg_img.jpg


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\bibtex}{\textsc{Bib}\!\TeX}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kms$}{ km s^{-1}}$
$\newcommand{$\bibtex$}{$\textsc${Bib}\!\TeX}$</div>



<div id="title">

# Multi-Scale Stellar Associations across the Star Formation Hierarchy in PHANGS--HST Nearby Galaxies: Methodology and Properties

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.11425-b31b1b.svg)](https://arxiv.org/abs/2212.11425)<mark>Appeared on: 2022-12-23</mark> - _Submitted to MNRAS. Referee report received with minor comments, and "request to clarify if the smaller associations are always included in the larger ones and how this may affect the photometric fitting of the larger association if the groups have different ages." Revision in progress_

</div>
<div id="authors">

Kirsten L. Larson, et al. -- incl., <mark><mark>Janice C. Lee</mark></mark>, <mark><mark>James Lilly</mark></mark>, <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Kathryn Kreckel</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** We develop a method to identify and determine the physical properties of stellar associations using\textit{Hubble Space Telescope}(\textit{HST})\textit{NUV-U-B-V-I}imaging of nearby galaxies from the PHANGS--HST survey.We apply a watershed algorithm to density maps constructed from point source catalogues Gaussian smoothed to multiple physical scales from 8 to 64 pc.We develop our method on two galaxies that span the distance range in the PHANGS--HST sample: NGC 3351 (10 Mpc), NGC 1566 (18 Mpc).We test our algorithm with different parameters such as the choice of detection band for the point source catalogue (\textit{NUV}or\textit{V}), source density image filtering methods, and absolute magnitude limits.We characterise the properties of the resulting multi-scale associations, including sizes, number of tracer stars, number of associations, photometry, as well as ages, masses, and reddening from Spectral Energy Distribution fitting.  Our method successfully identifies structures that occupy loci in the\textit{UBVI}colour--colour diagram consistent with previously published catalogues of clusters and associations. The median ages of the associations increases  from$\log (\mathrm{age/yr}) = 6.6$to$\log (\mathrm{age/yr}) = 6.9$as the spatial scale increases from 8 pc to 64 pc for both galaxies. We find that the youngest stellar associations, with ages$<3$Myr, indeed closely trace$\textsc${H ii}regions in H$\alpha$imaging, and that older associations are increasingly anti-correlated with the H$\alpha$emission.  Owing to our new method, the PHANGS--HST multi-scale associations provide a far more complete census of recent star formation activity than found with previous cluster and compact association catalogues.The method presented here will be applied to the full sample of 38 PHANGS--HST galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2212.11425/./lowRes_figs/NGC3351_reg_img.jpg" alt="Fig14" width="100%"/>

**Figure 14. -** An illustration of our watershed procedure for identifying stellar associations over multiple scales.  \textit{(Panel A)} An \textit{HST} three-colour image of NGC 3351 (Red: WFC3/UVIS F814W, Green: WFC3/UVIS F555W, Blue: WFC3/UVIS F438W+F336W+F275W).  \textit{(Panel B)} The \textit{NUV} image of the center of NGC 3351.
    \textit{(Panel C)}The \textit{NUV} image of the center of NGC 3351 with the position of all \textit{NUV} tracer stars shown as green points
    \textit{(Panel D)} The \mbox{16-pc} smoothed, filtered image of the \textit{NUV} tracer stars for the center of NGC 3351. The image scale is in units of the FWHM of the PSF brightness. The cutoff threshold is set to 1 FWHM and the \mbox{16-pc} watershed regions are shown in blue.
    \textit{(Panel E)}\textit{NUV} image of the center of NGC 3351 with 8, 16, 32, and \mbox{64-pc} levels overplotted as corresponding green, blue, purple, and orange regions. At $0.0396$ arcsec pixel$^{-1}$, one WFC3 pixel maps to $1.9$ pc pixel$^{-1}$ at a distance of $10$ Mpc for NGC 3351. (*FIG:ngc3351tracers*)

</div>
<div id="div_fig2">

<img src="tmp_2212.11425/./ngc3351_dolphot_color_vi_ub_ws_Mv_ALL.png" alt="Fig4" width="100%"/>

**Figure 4. -** Left: NGC 3351 UBVI colour--colour diagrams for stellar associations selected in the \textit{NUV}-band (left) and \textit{V}-band (right)for four scale levels (8, 16, 32, and 64 pc). The associations are coloured according to their absolute \textit{V}-band magnitude ($M_\mathrm{V}$). The Padova stellar models  are shown as grey points and the solar metallicity, $f_\mathrm{cov} = 0$ evolutionary tracks of synthetic stellar populations from BC03 are overplotted on the associations. The red line traces the path of a single-aged stellar population, while the yellow and blue lines trace composite stellar populations with $\tau = 2$ and $\tau = 4$, respectively.
     (*FIG:NGC3351_NUV_V_cc*)

</div>
<div id="div_fig3">

<img src="tmp_2212.11425/./lowRes_figs/NGC1566_reg_img.jpg" alt="Fig15" width="100%"/>

**Figure 15. -** Same as Figure \ref{FIG:ngc3351tracers}, except for NGC 1566. At $0.0396$ arcsec pixel$^{-1}$, one WFC3 pixel maps to $3.5$ pc pixel$^{-1}$ at a distance of $18$ Mpc for NGC 1566  \textit{(Panel A)} The \textit{HST} three-colour image for NGC 1566 (Red: WFC3/UVIS F814W, Green: WFC3/UVIS F555W, Blue: WFC3/UVIS F438W+F336W+F275W).
    \textit{(Panel B)} For illustration, we show a zoom-in of the \textit{NUV} image on a portion of the spiral arm with a bright star-forming region in NGC 1566.
    \textit{(Panel C)}The \textit{NUV} image of the star-forming region in NGC 1566 with the position of all \textit{NUV} tracer stars shown as green points.
    \textit{(Panel D)} The \mbox{16-pc} smoothed, filtered image of the \textit{NUV} tracer stars for the zoom in region of NGC 1566. The image scale is in units of the FWHM of the PSF brightness. The cutoff threshold is set to 1 FWHM and the \mbox{16-pc} watershed regions are shown in blue.
    \textit{(Panel E)}\textit{NUV} image of NGC 1566 with 16, 32, and \mbox{64-pc} levels overplotted as corresponding blue, purple, and orange regions. (*FIG:ngc1566tracers*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

146  publications in the last 7 days.
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers